definimos la estructura de un estado y una variable auxiliar que ayudara a otras funciones

In [1]:
cartas = [1,2,3,4,5,6,7,8,9,10,11,12,13]
struct estado
    cartas_en_mano::Int64
    valor_mano::Int64
    ases::Int64
    valor_dealer::Int64
    es_final::Int64
end

A continuacion definimos como son los estados finales y les asignamos 0 si no lo son, 1 si son finales positivos y -1 si son finales negativos

In [2]:
function isterminal(cartas,valor)
    terminal = 0
    if valor > 21
        terminal = -1
    else
        if valor == 21
            terminal = 1
        end
        if cartas == 4
            terminal = 1
        end
    end
    return terminal
end

isterminal (generic function with 1 method)

una simple funcion auxiliar para calcular el valor de la mano

In [3]:
function calcularValMano(mano)
    suma = 0
    aces = 0
    for i in mano
        if i == 1
            aces += 1
        end
        if i > 10
           suma += 10-i
        end
        suma += i
    end
    if aces > 0 && (suma + 10) <= 21
        suma += 10
    end
    suma,aces
end

calcularValMano (generic function with 1 method)

creamos todas las distintas formas distintas de los posibles estados existentes

In [4]:
estados = []
mano2cartas = []
mano3cartas = []
mano4cartas = []
for i in cartas
    for j in cartas
        for d in 4:26
            push!(mano2cartas,[i,j,d])
        end
    end
end

for i in cartas
    for j in cartas
        for k in cartas
            for d in 4:26
                push!(mano3cartas,[i,j,k,d])
            end
        end
    end
end

for i in cartas
    for j in cartas
        for k in cartas
            for l in cartas
                for d = 4:26
                    push!(mano4cartas,[i,j,k,l,d])
                end
            end
        end
    end
end

for i in mano2cartas
    a,b = calcularValMano(i[1:2])
    push!(estados,estado(2,a,b,i[3],isterminal(2,a)))
end
for i in mano3cartas
    a,b = calcularValMano(i[1:3])
    push!(estados,estado(3,a,b,i[4],isterminal(3,a)))
end
for i in mano4cartas
    a,b = calcularValMano(i[1:4])
    push!(estados,estado(4,a,b,i[5],isterminal(4,a)))
end


estados = unique(estados)
estados

3358-element Array{Any,1}:
 estado(2, 12, 2, 4, 0)  
 estado(2, 12, 2, 5, 0)  
 estado(2, 12, 2, 6, 0)  
 estado(2, 12, 2, 7, 0)  
 estado(2, 12, 2, 8, 0)  
 estado(2, 12, 2, 9, 0)  
 estado(2, 12, 2, 10, 0) 
 estado(2, 12, 2, 11, 0) 
 estado(2, 12, 2, 12, 0) 
 estado(2, 12, 2, 13, 0) 
 estado(2, 12, 2, 14, 0) 
 estado(2, 12, 2, 15, 0) 
 estado(2, 12, 2, 16, 0) 
 ⋮                       
 estado(4, 40, 0, 15, -1)
 estado(4, 40, 0, 16, -1)
 estado(4, 40, 0, 17, -1)
 estado(4, 40, 0, 18, -1)
 estado(4, 40, 0, 19, -1)
 estado(4, 40, 0, 20, -1)
 estado(4, 40, 0, 21, -1)
 estado(4, 40, 0, 22, -1)
 estado(4, 40, 0, 23, -1)
 estado(4, 40, 0, 24, -1)
 estado(4, 40, 0, 25, -1)
 estado(4, 40, 0, 26, -1)

In [5]:
estados_terminales = []
for i in estados
    if i.es_final != 0
        push!(estados_terminales,i)
    end
end
estados_terminales

2001-element Array{Any,1}:
 estado(2, 21, 1, 4, 1)  
 estado(2, 21, 1, 5, 1)  
 estado(2, 21, 1, 6, 1)  
 estado(2, 21, 1, 7, 1)  
 estado(2, 21, 1, 8, 1)  
 estado(2, 21, 1, 9, 1)  
 estado(2, 21, 1, 10, 1) 
 estado(2, 21, 1, 11, 1) 
 estado(2, 21, 1, 12, 1) 
 estado(2, 21, 1, 13, 1) 
 estado(2, 21, 1, 14, 1) 
 estado(2, 21, 1, 15, 1) 
 estado(2, 21, 1, 16, 1) 
 ⋮                       
 estado(4, 40, 0, 15, -1)
 estado(4, 40, 0, 16, -1)
 estado(4, 40, 0, 17, -1)
 estado(4, 40, 0, 18, -1)
 estado(4, 40, 0, 19, -1)
 estado(4, 40, 0, 20, -1)
 estado(4, 40, 0, 21, -1)
 estado(4, 40, 0, 22, -1)
 estado(4, 40, 0, 23, -1)
 estado(4, 40, 0, 24, -1)
 estado(4, 40, 0, 25, -1)
 estado(4, 40, 0, 26, -1)

segregamos los estados terminales 

In [6]:
estados = estados
estados_terminales = estados_terminales
acciones = ["hit","stand"]
descuento = .99
tau = .01
epsilon = .01
valores = []
politicas = []

0-element Array{Any,1}

definimos la funcion de probabilidad de transicion, donde vemos si es posible ir a un nuevo estado o no

In [7]:
function ptransicion(e::estado, accion, e_s::estado)
    if accion == "stand"
        if e == e_s
            return 1
        end
        if e.cartas_en_mano != e_s.cartas_en_mano
            return 0
        end
        if (e.valor_dealer - e_s.valor_dealer) <= 9 || (e.valor_dealer - e_s.valor_dealer) == 11
            return 1/13
        else
            return 4/13
        end
    #si la accion es hit
    else
        if (e.valor_mano - e_s.valor_mano) <= 9 || (e.valor_mano - e_s.valor_mano) == 11
            return 1/13
        else
            return 4/13
        end
    end
end

ptransicion (generic function with 1 method)

la recompensa la tenemos previamente definida en los estados y en la funcion de checar si algo es terminal

In [8]:
function recompensa(e::estado, accion, e_s::estado)
    e_s.es_final
end

recompensa (generic function with 1 method)

Iteracion por valores

In [9]:
function iterar_valores(estados, descuento, estados_finales, acciones)
    v = Dict(e => 0.0 for e in estados)
    v_p = Dict(e => 0.0 for e in estados)
    
    converge = false
    
    while !converge
        for e in keys(v)
            if e in estados_finales
                v[e] = recompensa(e, nothing, e)
            else
                v_p[e] = maximum([sum([ptransicion(e, a, e_s) * (recompensa(e, a, e_s) + descuento * v[e_s])
                                            for e_s in estados])
                                        for a in acciones])
            end

            converge = true
                            
            for e in keys(v)
                if v_p[e] > v[e]
                    v[e] = v_p[e]
                    converge = false
                end
            end
            
            if converge
                break
            end
        end
    end
    
    politica = Dict(e => acciones[1] for e in estados)
    
    for e in keys(v)
        actions_value = Dict(a => sum([ptransicion(e, a, e_s) * v[e_s] for e_s in estados])
                            for a in acciones)
                                
        politica[e] = findmax(actions_value)[2]
    end
    
    return politica
end

iterar_valores (generic function with 1 method)

In [10]:
politica = iterar_valores(estados,1,estados_terminales,acciones)


Dict{estado,String} with 3358 entries:
  estado(4, 13, 2, 10, 1)  => "stand"
  estado(4, 13, 1, 17, 1)  => "stand"
  estado(3, 6, 0, 23, 0)   => "hit"
  estado(4, 20, 1, 12, 1)  => "stand"
  estado(4, 24, 0, 6, -1)  => "stand"
  estado(2, 16, 1, 7, 0)   => "hit"
  estado(2, 14, 1, 23, 0)  => "hit"
  estado(2, 13, 1, 11, 0)  => "hit"
  estado(4, 20, 3, 11, 1)  => "stand"
  estado(3, 25, 0, 6, -1)  => "hit"
  estado(2, 16, 1, 9, 0)   => "hit"
  estado(3, 6, 0, 7, 0)    => "hit"
  estado(4, 26, 1, 25, -1) => "stand"
  estado(4, 15, 1, 11, 1)  => "stand"
  estado(2, 19, 1, 11, 0)  => "hit"
  estado(4, 18, 1, 25, 1)  => "stand"
  estado(4, 29, 1, 8, -1)  => "hit"
  estado(4, 36, 0, 5, -1)  => "hit"
  estado(3, 12, 0, 17, 0)  => "hit"
  estado(4, 12, 2, 22, 1)  => "stand"
  estado(4, 21, 2, 23, 1)  => "stand"
  estado(4, 20, 1, 14, 1)  => "stand"
  estado(3, 13, 1, 14, 0)  => "hit"
  estado(4, 27, 1, 17, -1) => "hit"
  estado(4, 19, 2, 9, 1)   => "stand"
  ⋮                        => ⋮